In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
df_bikes = pd.read_csv('bike_rentals_cleaned.csv')
X_bikes = df_bikes.iloc[:, :-1]
y_bikes = df_bikes.iloc[:, -1]
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [5]:
reg = DecisionTreeRegressor(random_state=2)
scores = cross_val_score(reg, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=5)

In [8]:
rmse = np.sqrt(-scores)
print(f'RMSE 평균: {rmse.mean():0.2f}')

RMSE 평균: 1233.36


In [10]:
reg = DecisionTreeRegressor()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_train)
from sklearn.metrics import mean_squared_error
reg_mse = mean_squared_error(y_train, y_pred)
reg_rmse = np.sqrt(reg_mse)
reg_rmse

0.0

train set에 대한 RMSE값이 0인 것으로 보아 과대적합되어 분산이 크다는 것을 알 수 있음

### 2.4.2 하이퍼파라미터

max_depth
- 트리의 깊이를 정의한다. (깊이는 분할 횟수를 결정)  
- max_depth의 기본 값은 None으로 제한이 없다. (과대적합을 만든다.)  
- max_depth를 작은 값으로 제한하면 분산이 줄어들고 모델이 새로운 데이터에 잘 일반화 됨.  

min_samples_leaf
- 리프 노드가 가질 수 있는 최소 샘플의 개수를 제한
- 기본값은 1로 제한이 없을 때 리프 노드는 하나의 샘플로 구성될 수 있다. (과대적합 되기 쉬움)

max_leaf_nodes
- 리프 노드의 샘플 개수를 지정하는 대신 리프 노드의 전체 개수를 지정하여 과대 적합을 막는다. 

max_features
- 분할마다 모든 특성을 고려하지 않고 매번 지정된 개수의 특성 중에서 선택한다. 
- 분산을 줄이는데 효과적인 매개변수이다. 

min_samples_split
- 분할하기 위해 필요한 최소 샘플 개수를 제한

min_impurity_decrease
- 분할하기 위한 최소 불순도 감소를 지정
  
GridSearchCV
- 교차 검증을 사용해 최선의 결과를 만드는 매개변수 조합을 찾는다.  
- GridSearchCV의 핵심은 매개변수 값의 딕셔너리를 만드는 것  

In [11]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth':[None, 2, 3, 4, 5, 6, 8, 10, 20]}

In [13]:
reg = DecisionTreeRegressor(random_state=2)
grid_reg = GridSearchCV(reg, params, scoring='neg_mean_squared_error', 
                       cv=5, return_train_score=True, n_jobs=-1)
grid_reg.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=2), n_jobs=-1,
             param_grid={'max_depth': [None, 2, 3, 4, 5, 6, 8, 10, 20]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [14]:
best_params = grid_reg.best_params_
print("최상의 매개변수:", best_params)

최상의 매개변수: {'max_depth': 6}


In [17]:
best_score = np.sqrt(-grid_reg.best_score_)
print(f'훈련 점수: {best_score:.3f}')

훈련 점수: 951.398


In [18]:
best_model = grid_reg.best_estimator_
y_pred = best_model.predict(X_test)
from sklearn.metrics import mean_squared_error 
rmse_test = mean_squared_error(y_test, y_pred)**0.5
print(f'테스트 점수: {rmse_test:.3f}')

테스트 점수: 864.670


## 2.5 심장 질환 예측하기 - 사례 연구
모델을 만든 후 심장 질환을 예측하는데 가장 중요한 특성을 알려주는 feature_importances_ 속성을 사용해 결과를 해석해본다. 

In [22]:
data_path = 'handson-gb-main/Chapter02/heart_disease.csv'
df_heart = pd.read_csv(data_path)
df_heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


cp: chest pain(1=전형적인 협심증, 2=비전형적인 협심증, 3=협심증이 아닌 통증, 4=무증상)  
trestbps: 안정혈압  
chol: 혈청 콜레스테롤  
fbs: 공복혈당  
restecg: 심전도 결과  
thalach: 최대 심장 박동수  
exang: 운동으로 인한 협심증  
oldpeak: 휴식 대비 운동으로 인한 ST 감소  
slope: 최대 운동 ST 세그먼트 기울기(1=상승 기울기, 2=수평, 3=하강 기울기)  
ca: 형광 투시로 착색된 주요 혈관 수(0-3)  
thal: 탈륨 스트레스 테스트, (3=정상, 6=고정 결함, 7=가역적 결함)  


In [23]:
X = df_heart.iloc[:, :-1]
y = df_heart.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [25]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=2)
scores = cross_val_score(model, X, y, cv=5)
print('정확도: ', np.round(scores, 2))
print(f'정확도 평균: {scores.mean():.2f}')

정확도:  [0.74 0.85 0.77 0.73 0.7 ]
정확도 평균: 0.76


RandomizedSearchCV
- 탐색할 하이퍼파라미터가 많을 때 GridSearchCV로 하이퍼파라미터 튜닝을 하면 너무 오랜 시간이 걸릴 수 있음
- GridSearchCV와 동일한 방식으로 동작하지만 모든 하이퍼파라미터 조합을 테스트하는 대신 랜덤한 조합을 테스트한다. 
- 모든 값을 테스트하지 않으며 제한된 시간 안에 최상의 조합을 찾는다. 

In [30]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
def randomized_search_clf(params, runs=20, clf=DecisionTreeClassifier(random_state=2)):
    rand_clf = RandomizedSearchCV(clf, params, n_iter=runs, cv=5, n_jobs=-1, random_state=2)
    # n_iter : 시도할 매개변수 조합의 횟수
    rand_clf.fit(X_train, y_train)
    best_model = rand_clf.best_estimator_
    best_score = rand_clf.best_score_
    print(f"훈련 점수: {best_score:.3f}")
    y_pred = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'테스트 점수: {accuracy:.3f}')
    return best_model

In [36]:
randomized_search_clf(
    params={
        'criterion':['entropy', 'gini'],
        'splitter':['random', 'best'],
        'min_samples_split':[2, 3, 4, 5, 6, 8, 10],
        'min_samples_leaf':[1, 0.01, 0.02, 0.03, 0.04],
        'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50, None], 
        'max_features':['auto', 0.95, 0.90, 0.85, 0.80, 0.75, 0.70],
        'max_depth':[None, 2, 4, 6, 8],
        'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075, 0.01, 0.05]
    })

훈련 점수: 0.758
테스트 점수: 0.789


DecisionTreeClassifier(criterion='entropy', max_depth=4, max_features=0.7,
                       max_leaf_nodes=15, min_samples_split=3,
                       min_weight_fraction_leaf=0.0025, random_state=2,
                       splitter='random')

In [37]:
best_model = randomized_search_clf(
    params={
        'min_samples_split':[2, 9, 10],
        'min_samples_leaf':[1, 0.035, 0.04, 0.045, 0.05],
        'max_leaf_nodes':[45, None], 
        'max_features':['auto', 0.78],
        'max_depth':[None, 6, 7],
        'min_weight_fraction_leaf':[0.0, 0.05, 0.06, 0.07],
        },
    runs=100)

훈련 점수: 0.802
테스트 점수: 0.868


In [38]:
scores = cross_val_score(best_model, X, y, cv=5)
print('정확도: ', np.round(scores, 2))
print(f'정확도 평균: {scores.mean():.2f}')

정확도:  [0.82 0.9  0.8  0.8  0.78]
정확도 평균: 0.82


### 2.5.5 특성 중요도
- 해당 머신러닝 모델에서 가장 중요한 특성을 확인
- 결정 트리는 feature_importances 속성을 제공한다. 

In [39]:
best_model.fit(X, y)

DecisionTreeClassifier(max_depth=7, max_features=0.78, max_leaf_nodes=45,
                       min_samples_leaf=0.045, min_samples_split=9,
                       min_weight_fraction_leaf=0.06, random_state=2)

In [40]:
best_model.feature_importances_

array([0.04826754, 0.04081653, 0.48409586, 0.00568635, 0.        ,
       0.        , 0.        , 0.00859483, 0.        , 0.02690379,
       0.        , 0.18069065, 0.20494446])

In [41]:
feature_dict = dict(zip(X.columns, best_model.feature_importances_))
import operator
sorted(feature_dict.items(), key=operator.itemgetter(1), reverse=True)[0:3]

[('cp', 0.4840958610240171),
 ('thal', 0.20494445570568706),
 ('ca', 0.18069065321397942)]